In [7]:
from architectural_principles import ArchitecturalConstraints, State
from train import ExperimentManager
from environment import ArchitecturalEnvironment

In [6]:
config = {
    'env': {
        'grid_size': (10, 10),
        'max_steps': 500,
        'required_rooms': ArchitecturalConstraints.default_rooms()
    },
    'algorithms': {
        'value_iteration': {
            'gamma': .95,
            'theta': 0.001
        }
    }
}

In [3]:
def_rooms = ArchitecturalConstraints.default_rooms()

In [ ]:
import torch
import numpy as np
import concurrent.futures
from torch.multiprocessing import Pool


def generate_room_dimensions_for_room(room_ranges_items: tuple):
        """
        Generate the dimensions (width, height) combinations for a single room.
        This is a helper function that can be run in parallel.
        """
        room, (w_range, h_range) = room_ranges_items
        widths = np.arange(w_range[0], w_range[1] + 1)
        heights = np.arange(h_range[0], h_range[1] + 1)
        # Create a mesh grid of all width and height combinations
        w_grid, h_grid = np.meshgrid(widths, heights)
        # Stack the grids to get a list of (w, h) tuples
        room_dimensions = np.vstack([w_grid.ravel(), h_grid.ravel()]).T
        return room, room_dimensions


# Check overlap using batched parallel computation
def check_overlaps(positions_batch):
    batch_size = positions_batch.size(0)
    grid = torch.zeros((batch_size, G_width, G_height), dtype=torch.int32, device='cuda')  # Use int32 for counting

    # Iterate over the batch and unpack each room's coordinates (x, y, w, h)
    for i in range(batch_size):
        room = positions_batch[i]  # Now room is a tensor of shape [4] (x, y, w, h)
        
        # Ensure that room is a 1D tensor (x, y, w, h)
        if room.dim() != 1 or room.size(0) != 4:
            print(f"Unexpected room shape: {room.shape}")
            continue

        x, y, w, h = room[0].item(), room[1].item(), room[2].item(), room[3].item()  # Get scalar values

        # Fill grid for each configuration in batch
        for bx in range(x, x + w):
            for by in range(y, y + h):
                grid[i, bx, by] += 1  # Increment cells for each room's area

    # Valid if all grid cells are <= 1
    return (grid <= 1).all(dim=(1, 2))  # Check if no cell is occupied by more than 1 room




def generate_room_configs_parallel(room_ranges: dict, grid_size: tuple):
    G_width, G_height = grid_size

    # Create ranges for each room type
    room_types = list(room_ranges.keys())

    def generate_room_dimensions_parallel(room_ranges):
        """
        Generate room dimensions for all rooms in parallel.
        """
        dimensions = {}        
        # Use ProcessPoolExecutor to parallelize the generation of dimensions
        executor = concurrent.futures.ThreadPoolExecutor()
        for result in executor.map(generate_room_dimensions_for_room, room_ranges.items()):
            room, room_dimensions = result
            dimensions[room] = room_dimensions
        return dimensions
    
    dimensions = generate_room_dimensions_parallel(room_ranges)
    # Generate all possible positions for a room given its dimensions
    def generate_positions(dimensions):
        positions = []
        for w, h in dimensions:
            for x in range(G_width - w + 1):
                for y in range(G_height - h + 1):
                    positions.append((x, y, w, h))
        return positions

    # Generate position tensors for all rooms
    all_positions = [torch.tensor(generate_positions(dimensions[room]), dtype=torch.int32, device='cuda') for room in room_types]
    all_positions_flattend = [pos.flatten() for pos in all_positions]
    product = torch.meshgrid(*all_positions_flattend, indexing='ij')
    all_batches = torch.stack(product, dim=-1).reshape(-1, len(all_positions_flattend))
    print("batched")
    with Pool() as pool:
        results = pool.map(check_overlaps, all_batches)

    # Combine all room positions in parallel, iterate over all possible combinations of room placements
    valid_combinations = []
    
    # Generate all possible combinations of positions for each room
    from itertools import product
    for batch in product(*all_positions):
        batch_tensor = torch.stack(batch, dim=0).to('cuda')  # Stack into a single tensor of shape (num_rooms, num_positions, 4)
        # Check for overlap, ensure the result is True for all rooms in the batch
        if check_overlaps(batch_tensor).all():  # .all() ensures the batch is valid
            valid_combinations.append(batch_tensor.cpu().tolist())
        
    return valid_combinations


# Example usage
room_ranges = {
    "R1": ((2, 4), (2, 4)),
    "R2": ((1, 3), (1, 3)),
#     "R3": ((3, 5), (3, 5)),
#     "R4": ((2, 3), (2, 3)),
#     "R5": ((1, 2), (1, 2)),
#     "R6": ((2, 5), (1, 4))  # Added a 6th room type
}
grid_size = (10, 10)
states = generate_room_configs_parallel(room_ranges, grid_size)
print(f"Total valid configurations: {len(states)}")


batched


In [1]:
import torch
import random

def approximate_value_iteration(room_ranges, grid_size, num_samples, gamma=0.9, num_iterations=100):
    """
    Approximate the state space and perform value iteration.

    Parameters:
        room_ranges (dict): Dictionary mapping room names to dimension ranges ((w_min, w_max), (h_min, h_max)).
        grid_size (tuple): Dimensions of the grid (width, height).
        num_samples (int): Number of sampled states.
        gamma (float): Discount factor for value iteration.
        num_iterations (int): Number of value iteration steps.

    Returns:
        torch.Tensor: Approximated value function for sampled states.
        list: Sampled states.
    """
    G_width, G_height = grid_size

    # Sample states randomly
    def sample_random_state():
        state = []
        for room, ((w_min, w_max), (h_min, h_max)) in room_ranges.items():
            w = random.randint(w_min, w_max)
            h = random.randint(h_min, h_max)
            x = random.randint(0, G_width - w)
            y = random.randint(0, G_height - h)
            state.append((x, y, w, h))
        return state

    sampled_states = [sample_random_state() for _ in range(num_samples)]

    # Initialize value function
    values = torch.zeros(num_samples, device='cuda')

    # Define reward function (example: penalize overlap, encourage coverage)
    def compute_reward(state):
        grid = torch.zeros((G_width, G_height), dtype=torch.bool, device='cuda')
        reward = 0
        for x, y, w, h in state:
            if torch.any(grid[x:x+w, y:y+h]):  # Overlap penalty
                reward -= 10
            else:
                reward += w * h  # Reward based on area
                grid[x:x+w, y:y+h] = True
        return reward

    # Compute transition function (here we approximate with random transitions)
    def transition(state):
        # Slightly perturb the state
        new_state = []
        for x, y, w, h in state:
            x = max(0, min(G_width - w, x + random.randint(-1, 1)))
            y = max(0, min(G_height - h, y + random.randint(-1, 1)))
            new_state.append((x, y, w, h))
        return new_state

    # Value iteration
    for _ in range(num_iterations):
        new_values = torch.zeros_like(values)
        for i, state in enumerate(sampled_states):
            reward = compute_reward(state)
            next_state = transition(state)

            # Find index of the next state in sampled_states (approximation)
            try:
                next_index = sampled_states.index(next_state)
                next_value = values[next_index]
            except ValueError:
                next_value = 0  # Default value for unrecognized states

            new_values[i] = reward + gamma * next_value

        values = new_values

    return values, sampled_states

# Example usage
room_ranges = {
    "R1": ((2, 4), (2, 4)),
    "R2": ((1, 3), (1, 3)),
    "R3": ((3, 5), (3, 5)),
    "R4": ((2, 3), (2, 3)),
    "R5": ((1, 2), (1, 2)),
    "R6": ((1, 2), (1, 2))
}
grid_size = (10, 10)
num_samples = 1000

values, states = approximate_value_iteration(room_ranges, grid_size, num_samples)
print("Approximated Value Function:", values)
print("Sampled States:", states)


Approximated Value Function: tensor([ 17.,  -1.,  12.,   3., -12.,   1.,  12.,  36.,  21.,  -5.,   6.,  26.,
         22.,  11.,  30.,  -5.,  -4.,  -7.,  14.,  17.,  15.,  20.,  24.,  21.,
         11.,  10.,  -8.,   4.,  -2.,  12.,   9.,  20.,  30.,  11.,  16.,   5.,
         16.,  -1., -22.,  20., -18.,  17.,   5.,   8.,  16.,  13.,  -2., -12.,
          5.,  32.,  13.,  -6.,   1.,   1., -30.,   8.,  11.,   3.,  18.,  33.,
        -19.,  28.,   0.,  18.,  -8., -18.,  -6.,  -9.,  -8.,  28., -15.,  -7.,
         -9.,  -6.,  32.,   6.,  21.,  28.,  14.,  22.,  -7.,  11.,  11.,   1.,
         23.,  -1.,   9.,  12.,   1.,  12.,  -2.,  12.,  12.,  22.,  21.,  15.,
         22.,   0.,  -2.,  27.,  -9.,   6.,   0., -34.,  12.,   7.,  -6.,  -2.,
        -10.,  14.,  15.,  -1.,  10.,  13.,  16.,  38.,  16.,  24., -14., -15.,
          0., -15.,  21.,  40.,  -4.,   5.,  25.,  23.,  17.,  -6.,  11.,  13.,
         35.,   5.,   0.,   6.,  21.,  15., -11.,  19.,  19.,  12.,  -2.,   0.,
         13

In [4]:
env = ArchitecturalEnvironment((10, 10), 50, def_rooms)

In [ ]:
def _random_new_states(state)

In [ ]:
init_state = env.reset()
env.step()

In [9]:
init_state

State(layout=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), placed_rooms={}, current_step=0, required_rooms={'living': RoomRequirements(room_type=<RoomType.LIVING: 'living'>, min_size=(4, 4), max_size=(6, 6), privacy_level=<PrivacyLevel.PUBLIC: 1>, needs_natural_light=True, needs_ventilation=False, adjacent_to={<RoomType.ENTRY: 'entry'>, <RoomType.DINING: 'dining'>}, min_distance_from=None), 'bedroom': RoomRequirements(room_type=<RoomType.BEDROOM: 'bedroom'>, min_size=(3, 3), max_size=(4, 4), privacy_level=<PrivacyLevel.PRIVATE: 3>, needs_natural_light=True, needs_ventilation=False, adjacent_to=None, min_distance_from={<RoomType.ENTRY: 'entry'>, <

In [15]:
import numpy as np
import copy
init_state
# Initialize state-value map and sampled states
state_value_map = {}
sampled_states = [init_state]
num_iterations = 30
max_states = 1000
gamma = 0.95

def state_to_key(state) -> str:
    """Convert state to a hashable key."""
    return str(state.layout) + str(state.placed_rooms) + str(state.current_step)

def sample_initial_states():
    """Generate initial random states."""
    states = []
    env.reset()
    for room_name in env.required_rooms.keys():
        action = {
            "type": "add_room",
            "params": {
                "name": room_name,
                "room_type": env.required_rooms[room_name].room_type,
                "position": (
                    np.random.randint(0, env.grid_size[0] - 1),
                    np.random.randint(0, env.grid_size[1] - 1)
                ),
                "size": env.required_rooms[room_name].min_size,
            },
        }
        env.step(action)
        states.append(env._get_state())
    return states

# Sample initial states
sampled_states = sample_initial_states()
for state in sampled_states:
    state_value_map[state_to_key(state)] = 0.0

for _ in range(num_iterations):
    new_state_value_map = copy.deepcopy(state_value_map)

    # Expand states using valid actions
    expanded_states = []
    for state in sampled_states:
        env.set_state(state)
        for room_name in env.required_rooms.keys():
            # Generate random valid actions
            position = (
                np.random.randint(0, env.grid_size[0] - 1),
                np.random.randint(0, env.grid_size[1] - 1),
            )
            #### loop through actions here
            action = {
                "type": "add_room",
                "params": {
                    "name": room_name,
                    "room_type": env.required_rooms[room_name].room_type,
                    "position": position,
                    "size": env.required_rooms[room_name].min_size,
                },
            }
            next_state, reward, _, _ = env.step(action)

            # Add new state if it wasn't visited before
            key = state_to_key(next_state)
            if key not in state_value_map and len(state_value_map) < max_states:
                state_value_map[key] = 0.0
                expanded_states.append(next_state)

    # Update value function
    for state in sampled_states:
        env.set_state(state)
        max_next_value = 0
        for room_name in env.required_rooms.keys():
            # Try adding one room
            position = (
                np.random.randint(0, env.grid_size[0] - 1),
                np.random.randint(0, env.grid_size[1] - 1),
            )
            action = {
                "type": "add_room",
                "params": {
                    "name": room_name,
                    "room_type": env.required_rooms[room_name].room_type,
                    "position": position,
                    "size": env.required_rooms[room_name].min_size,
                },
            }
            next_state, reward, _, _ = env.step(action)
            key = state_to_key(next_state)
            max_next_value = max(max_next_value, reward + gamma * state_value_map.get(key, 0))

        # Update the value of the current state
        new_state_value_map[state_to_key(state)] = max_next_value

    # Replace old value map with the new one
    state_value_map = new_state_value_map

    # Deduplicate states and merge expanded states
    sampled_states.extend(expanded_states)
    sampled_states = list({state_to_key(s): s for s in sampled_states}.values())
    
state_value_map

{"[[0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]]{'bedroom': {'id': 1, 'type': <RoomType.BEDROOM: 'bedroom'>, 'position': (3, 0), 'size': (3, 3)}, 'bathroom': {'id': 2, 'type': <RoomType.BATHROOM: 'bathroom'>, 'position': (2, 3), 'size': (2, 2)}, 'entry': {'id': 3, 'type': <RoomType.ENTRY: 'entry'>, 'position': (3, 8), 'size': (2, 2)}}1": 0.0,
 "[[0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [1 1 1 0 0 0 0 0 0 0]\n [1 1 1 0 0 0 0 0 0 0]\n [1 1 1 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0 0]]{'bedroom': {'id': 1, 'type': <RoomType.BEDROOM: 'bedroom'>, 'position': (3, 0), 'size': (3, 3)}, 'bathroom': {'id': 2, 'type': <RoomType.BATHROOM: 'bathroom'>, 'position': (2, 3), 'size': (2, 2)}, 'entry': {'id': 3, 'typ